# Implementing AlexNet with Keras

I will be implementing the AlexNet CNN architecture with Keras in python. I will not be using TensorFlow backend since my machine has an AMD GPU, thus i will be using PlaidML backend. I will also train the model on CPU just to show how bad it is. The dataset will be CIFAR-10 which has 60k images for 10 classes.

## PlaidML as backend instead of TensorFlow

To use PlaidML as backend follow these steps
- activate project environment
- `plaidml-setup`
- `Enable experimental device support? (y,n)[n]:n`
- `Please choose a default device:` choose 1,2,3.. most probably 1 would be your CPU, 2 would be the integrated GPU of your CPU and 3 would be your dedicated GPU
- `Save settings to /Users/veersingh/.plaidml? (y,n)[y]:y`

In [1]:
# import libraries and set backend to plaidML
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

Using plaidml.keras.backend backend.


## Processing the data

First we specify the paths to the directories containing our training and testing sets. Next we will rescale the images from 0-255 to 0-1 to reduce computations. Then we will use the flow_from_directory() method of the ImageDataGenerator class to resize the image to 227x227x3. This is important since AlexNet expects the input image to be of size 227x227 with 3 channels or rgb. Our input images are of the size 32x32x3 which means we will have to upscale them to 227x227 and keep the channels same.

In [2]:
# Specify the paths
train_dataset_path = '/Users/veersingh/Desktop/cifar_10/train'
test_dataset_path = '/Users/veersingh/Desktop/cifar_10/test'

In [3]:
# Rescale
train = ImageDataGenerator(rescale=1 / 255)
test = ImageDataGenerator(rescale=1 / 255)

# Define train and test dataset
# This method is useful since our images are in their respective class folders

# seed = random number to maintain same output everytime
# directory = path of train or test dataset
# target_size = resize the input image to required shape
# batch_size = default batch size of 32
# class_mode = categorical for multiclass
# color_mode = depending on number of channels required by model, grayscale for 1 channel, rgb for 3 channel

train_dataset = train.flow_from_directory(seed=1,
                                          directory=train_dataset_path,
                                          target_size=(227, 227),
                                          batch_size=32,
                                          class_mode = 'categorical',
                                          color_mode='rgb')

test_dataset = test.flow_from_directory(seed=1,
                                        directory=test_dataset_path,
                                        target_size=(227, 227),
                                        batch_size=32,
                                        class_mode = 'categorical',
                                        color_mode='rgb')

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Create the Model

In [4]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(227,227,3), padding='valid'))

# Pooling 
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))

# Pooling
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))

# Pooling
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Flatten
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096, activation='relu'))

# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096, activation='relu'))

# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

#  output Layer, AlexNet has a 1000 neurons in the last FC layer but we have 10 classes, thus we will set it to 10
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


In [5]:
# Get model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       885120    
__________

## Train the model

We will train the model for 30 epochs. An epoch is the number of times that the learning algorithms runs through the entire training dataset. Increasing this number will increase the training time.

In [ ]:
# Train the model

# start the timer
start_time = time.time()


history = model.fit_generator(train_dataset,
                              steps_per_epoch=1560, # number of training images/batch size = 50000/32 = 1562
                              epochs=2,
                              validation_data=test_dataset, # validate against the test set on each epoch
                              verbose=2) # print constant updates

# optionally save the model
# model.save('alexnet_cifar_10.h5')

# stop the timer
time_elapsed = time.time()-start_time
print(f'The model took {time_elapsed}' )